# Cases of concepts in LNN

**_Jingren Wang, Oct 11th, 2025_**

## The representation of the "Cat-Dog example"
An example on left part covering upward, downward, etc.
<center>
    <figure>
    <img src="CatDogExample.png" width="330" />
        <figcaption>Figure 1. Cat-Dog example on LNN.</figcaption>
 </figure>
</center>

In [154]:
from lnn import Proposition, And, Or, Fact, Implies

# Rules
Whiskers = Proposition("Whiskers")
Tail = Proposition("Tail")
Cat = Proposition("Cat")
Dog = Proposition("Dog")
Chases = Proposition("Chases")
Pet = Proposition("Pet")
LaserPointer = Proposition("LaserPointer")

# You saw a laser pointer, you knew the implying rules
LaserPointer.add_data(Fact.TRUE)
IMPLIES = Implies(LaserPointer, Chases)
IMPLIES.add_data(Fact.TRUE)
# Reasoning(D) the result(We don't really need this in this example, just showcase here)
IMPLIES.downward()
print("== Show the intermidate data on Chases ==\n")
Chases.print()

== Show the intermidate data on Chases ==

OPEN Proposition: Chases                                    TRUE (1.0, 1.0)



In [155]:
Whiskers.add_data(Fact.TRUE)
Tail.add_data(Fact.TRUE)
#AND = And(Whiskers, Tail)
#AND.upward()
AND2 = And(Whiskers, Tail, IMPLIES)
# Reasoning(U) the result
AND2.upward()
print("\n== Show the intermidate data on left AND2 ==\n")
AND2.print()



== Show the intermidate data on left AND2 ==

OPEN And: (Whiskers ∧ Tail ∧ (LaserPointer → Chases))          TRUE (1.0, 1.0)



In [156]:
Cat.add_data(Fact.TRUE)
# Reasoning(D) the result on IMPLES2
IMPLIES2 = Implies(AND2, Cat)
IMPLIES2.upward()
print("\n== Show the intermidate data on IMPLES2 ==\n")
IMPLIES2.print()


== Show the intermidate data on IMPLES2 ==

OPEN Implies: ((Whiskers ∧ Tail ∧ (LaserPointer → Chases)) → Cat)          TRUE (1.0, 1.0)



**_It's okey if an input of an OR is undefined:_**

In [157]:
OR_Animal = Or(Cat, Dog)
OR_Animal.upward()
print("\n== Show the intermidate data on OR_Animal ==\n")
OR_Animal.print()


== Show the intermidate data on OR_Animal ==

OPEN Or: (Cat ∨ Dog)                                        TRUE (1.0, 1.0)



In [158]:
IMPLIES3 = Implies(OR_Animal, Pet)
Pet.add_data(Fact.FALSE)
IMPLIES3.upward()
print("\n== Show the intermidate data on IMPLES3 ==\n")
IMPLIES3.print()


== Show the intermidate data on IMPLES3 ==

OPEN Implies: ((Cat ∨ Dog) → Pet)                          FALSE (0.0, 0.0)



### An unknown case

In [159]:
IMPLIES_u = Implies(LaserPointer, Chases)
print("== Show the possiblilty intermidate data on IMPLIES_p ==\n")
IMPLIES_u.print()

== Show the possiblilty intermidate data on IMPLIES_p ==

OPEN Implies: (LaserPointer → Chases)                    UNKNOWN (0.0, 1.0)



### A simple showcase on possibilities
You could try to fully edit the inputs after this talk to make a concrete example.

In [160]:
LaserPointer.add_data((.8, .9))
Chases.add_data((.1, .2))
IMPLIES_p = Implies(LaserPointer, Chases)
IMPLIES_p.upward()
print("== Show the possiblilty intermidate data on IMPLIES_p ==\n")
IMPLIES_p.print()

== Show the possiblilty intermidate data on IMPLIES_p ==

OPEN Implies: (LaserPointer → Chases)               APPROX_FALSE (0.2, 0.4)



# LNN Training (Borrowed from IBM LNN)

In this section, we demonstrate how to train a single logical conjunction to become consistent when inconsistencies arise. 

If we are given a mix of _True_ and _False_ propositions in a logical conjunction that we expected to also be _True_:
<center>
    <figure>
    <img src="./img/2/and_1.png" width="520" />
        <figcaption>Figure 1. Logical conjunction between nodes</figcaption>
 </figure>
</center>

In [161]:
from lnn import Propositions, And, Fact

# Rules
A, B, C, D, E = Propositions("A", "B", "C", "D", "E")
AND = And(A, B, C, D, E)


# Data
A.add_data(Fact.TRUE)
B.add_data(Fact.FALSE)
C.add_data(Fact.TRUE)
D.add_data(Fact.FALSE)
E.add_data(Fact.TRUE)
AND.add_data(Fact.TRUE)

We expect an upward inference at the conjunction will cause a contradiction:
<center>
    <figure>
    <img src="./img/2/and_2.png" width="520" />
        <figcaption>Figure 2. Upward inference causing a contradiction</figcaption>
 </figure>
</center>

In [162]:
from lnn import Model, Loss, Direction

# Knowledge
model = Model()
model.add_knowledge(AND)

# Reasoning
model.infer()
AND.print()

OPEN And: (A ∧ B ∧ C ∧ D ∧ E)                      CONTRADICTION (1.0, 0.0)



By introducing a loss on the contradiction:

\begin{align}
 Loss = \lambda \sum_{\forall i} \text{max} (0, L_i - U_i)
\end{align}

we can train the LNN to adjust the weights such that all sources of contradictions (the _False_ inputs) become downweighted and eventually removed.

<center>
    <figure>
    <img src="./img/2/and_3.png" width="520" />
        <figcaption>Figure 3. Training the LNN using a contradiction loss</figcaption>
 </figure>
</center>


A final reasoning pass with the updated parameters causes the logical conjunction to remain <strong style=color:#0e6027>TRUE</strong> and keeps it consistent with the data that has been supplied. 


In [163]:
# Training
model.train(direction=Direction.UPWARD, losses=[Loss.CONTRADICTION])
AND.print(params=True)

OPEN And: (A ∧ B ∧ C ∧ D ∧ E)                               TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 0. 1. 0. 1.]


As expected, the weights of the `False` inputs have all dropped to zero, removing their contribution to the truth of the `And` node. The resulting network therefore reasons in a self-supervised manner, ensuring that the interaction between the knowledge and the data is consistent.
> When unspecified, the loss coefficient $\lambda = 1$